In [546]:
import pandas as pd
import numpy as np
import random
import joblib
import seaborn as sns
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import label_binarize, LabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline

In [547]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

#### The data

In [548]:
#df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_4_missing_1.csv')
train_df = pd.read_csv('../../final/data/train_set_basic.csv')
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,9.007012,-1.000000,-1.000000,3.519565,440.499323,103.442762,59.017997,2.612173,1,0.650757,114.794964,112.308159,25.612786,5.969710,116.026042,27.021037,13.397977
1,8.760976,-1.000000,0.491469,-1.000000,259.895852,103.885481,-1.000000,2.529991,0,0.728641,74.824352,-1.000000,-1.000000,-1.000000,-1.000000,26.282929,-1.000000
2,7.490324,70.812609,-1.000000,1.495604,482.109919,79.543391,-1.000000,2.824995,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,22.470972,-1.000000
3,8.113370,499.313483,3.507612,0.000000,195.351883,100.545858,203.895756,2.420797,1,0.860863,9.120947,41.131511,1.843890,18.845473,106.199806,24.340111,104.373581
4,13.935301,349.569415,5.190725,6.894195,489.595939,102.234294,150.085853,4.089225,1,0.216907,20.344863,92.547095,19.815123,29.543875,98.388710,41.805903,30.655044


In [549]:
# class_dict = constants.CLASS_DICT
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]
# X_train, X_test_fake, y_train, y_test_fake = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)

In [550]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000


In [551]:
cols = list(X_test.columns)

In [553]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [554]:
dt = DecisionTreeClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred = dt.predict(X_test)
test_df = pd.DataFrame()
test_df['y_actual'] = y_test
test_df['y_pred'] = y_pred
test_df.isna().sum()

C:\Users\User\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



y_actual    0
y_pred      0
dtype: int64

In [555]:
dot_data = export_graphviz(dt, out_file='../../final/images/dt_basic.dot', 
                           feature_names = cols,  
                           class_names = list(constants.CLASS_DICT.keys()),
                           filled=True)
# Convert to image using dot -Tpng images/dt.dot -o images/dt.png 
# Make sure to have graphviz installed (windows executable file lie normal software)

In [556]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

99.94285714285715

In [511]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.7703571428571429, 0.7506663379246592, 0.8593844877213418)

In [512]:
test_df.y_pred.unique()

array([5., 1., 4., 0., 2., 3., 6., 7.])

In [513]:
test_df.y_actual.unique()

array([5, 1, 4, 0, 7, 2, 3, 6], dtype=int64)

In [514]:
# test_df.to_csv('../../final/test_dfs/dt_noisy_4_missing_5.csv', index=False)

#### Without inconclusive diagnosis

In [515]:
test_df_sans_inconc = test_df[~((test_df.y_actual == 7.0) | (test_df.y_pred ==7.0))]

In [516]:
test_df_sans_inconc.y_actual.unique()

array([5, 1, 4, 0, 2, 3, 6], dtype=int64)

In [517]:
test_df_sans_inconc.y_pred.unique()

array([5., 1., 4., 0., 2., 3., 6.])

In [518]:
success_rate, success_df = utils.success_rate(test_df_sans_inconc)
success_rate

82.50355618776672

#### Feature Importance

In [519]:
# utils.compute_feature_importance(dt, X_train)

#### Using SHAP

In [520]:
# import shap

##### delete from here

In [521]:
# shap.initjs()

# #background = X_train.iloc[0:100,:].values.astype('float')
# background = X_train.values.astype('float')
# explainer = shap.KernelExplainer(dt.predict_proba, background)

# all_shap_values = explainer.shap_values(X_test)

In [522]:
# shap.force_plot(explainer.expected_value[6], row_shap_values[6], row_for_prediction, feature_names=X_train.columns)

In [523]:
# row_for_prediction = X_test.iloc[1:2,:].values.astype('float')
# row_shap_values = explainer.shap_values(row_for_prediction)
# shap.summary_plot(row_shap_values, row_for_prediction, feature_names = list(X_test.columns))

In [524]:
# len(list(X_test.columns))

In [525]:
# shap.summary_plot(shap_values, X_test)

In [526]:
# shap_values[0]

##### end here

In [527]:
# explainer = shap.TreeExplainer(dt)
# shap_values = explainer.shap_values(X_test)
# len(shap_values)

In [528]:
# shap_values[0].shape

In [529]:
# shap.summary_plot(shap_values, X_test)

In [530]:
# def get_ABS_SHAP(df_shap,df):
#     #import matplotlib as plt
#     # Make a copy of the input data
#     shap_v = pd.DataFrame(df_shap)
#     feature_list = df.columns
#     shap_v.columns = feature_list
#     df_v = df.copy().reset_index().drop('index',axis=1)
    
#     # Determine the correlation in order to plot with different colors
#     corr_list = list()
#     for i in feature_list:
#         b = np.corrcoef(shap_v[i],df_v[i])[1][0]
#         corr_list.append(b)
#     corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
 
#     # Make a data frame. Column 1 is the feature, and Column 2 is the correlation coefficient
#     corr_df.columns  = ['Variable','Corr']
#     corr_df['Sign'] = np.where(corr_df['Corr']>0,'red','blue')
    
#     shap_abs = np.abs(shap_v)
#     k=pd.DataFrame(shap_abs.mean()).reset_index()
#     k.columns = ['Variable','SHAP_abs']
#     k2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
#     k2 = k2.sort_values(by='SHAP_abs',ascending = True)
    
#     k2_f = k2[['Variable', 'SHAP_abs', 'Corr']]
#     k2_f['SHAP_abs'] = k2_f['SHAP_abs'] * np.sign(k2_f['Corr'])
#     k2_f.drop(columns='Corr', inplace=True)
#     k2_f.rename(columns={'SHAP_abs': 'SHAP'}, inplace=True)
    
#     return k2_f


In [531]:
# foo_all = pd.DataFrame()

# for k,v in list(enumerate(dt.classes_)):
#     foo = get_ABS_SHAP(shap_values[k], X_test)
#     foo['class'] = v
#     foo_all = pd.concat([foo_all,foo])

# import plotly_express as px
# px.bar(foo_all,x='SHAP', y='Variable', color='class')

#### Using LIME

In [532]:
# import lime
# from lime import lime_tabular

In [533]:
# lime_explainer = lime_tabular.LimeTabularExplainer(training_data=np.array(X_train), feature_names=X_train.columns, 
#                                               class_names=list(class_dict.keys()), mode='classification')

In [534]:
# row_for_prediction = row_for_prediction.reshape(18,)

In [535]:
# exp = lime_explainer.explain_instance(row_for_prediction, dt.predict_proba)
# exp.show_in_notebook(show_table=True)

In [536]:
# y_test.iloc[1:2]

In [537]:
# exp = explainer.explain_instance(data_row=X_test.iloc[1],  predict_fn=dt.predict_proba, num_features=5)
# exp.show_in_notebook(show_table=True)

In [538]:
# X_test.iloc[39]